In [106]:
import torch
import numpy as np
import pandas as pd
import lightning as L
import torch.nn as nn
import torch.nn.functional as F
import sklearn
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
import torch.utils
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm


In [107]:
if (torch.cuda.is_available()):
    device = torch.device('cuda')
else:
    device = torch.device('cpu')



In [108]:
data = pd.read_csv('/Users/arsenijdzeban/Downloads/creditcard_2023.csv')

In [109]:
data.head()

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.10,0
1,1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.37,0
2,2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.54,0
3,3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.44,0
4,4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.97,0


In [110]:
AmountScaler = sklearn.preprocessing.MinMaxScaler(feature_range=(-1,1))

In [111]:
X = data.drop(columns=['id', 'Class'])
y = data['Class'].to_numpy().reshape(-1,1)

In [112]:
X_scaled = AmountScaler.fit_transform(X)

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=True)

In [114]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = torch.tensor(x, dtype= torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [115]:
train_data = MyDataset(X_train, y_train)
test_data = MyDataset(X_test, y_test)

In [116]:
batch_size = 128
testloader = DataLoader(test_data, batch_size=batch_size, shuffle = False, drop_last=True)
trainloader = DataLoader(train_data, batch_size=batch_size, shuffle = True)

In [117]:
input_size = len(X.values[0])

In [118]:
class LinearNet(nn.Module):
    def __init__(self, input_size):
        super(LinearNet,self).__init__()
        self.layers = nn.Sequential(
        nn.Linear(input_size,128),
        nn.ReLU(),
        nn.Linear(128, 1),
        nn.Sigmoid()
        )
        
    def forward(self, x):
        return  self.layers(x)

In [119]:
model = LinearNet(input_size=input_size).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [120]:
epochs = 100
for epoch in tqdm(range(epochs), desc='Training'):
    epoch_loss =0.0
    model.train()

    for input, target in trainloader:
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        epoch_loss+=loss.item()
    
    avg_loss = epoch_loss/len(trainloader)

    tqdm.write(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f}")

Training:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1/100 | Loss: 0.0998
Epoch 2/100 | Loss: 0.0624
Epoch 3/100 | Loss: 0.0503
Epoch 4/100 | Loss: 0.0438
Epoch 5/100 | Loss: 0.0392
Epoch 6/100 | Loss: 0.0385
Epoch 7/100 | Loss: 0.0371
Epoch 8/100 | Loss: 0.0355
Epoch 9/100 | Loss: 0.0323
Epoch 10/100 | Loss: 0.0308
Epoch 11/100 | Loss: 0.0300
Epoch 12/100 | Loss: 0.0284
Epoch 13/100 | Loss: 0.0273
Epoch 14/100 | Loss: 0.0276
Epoch 15/100 | Loss: 0.0266
Epoch 16/100 | Loss: 0.0266
Epoch 17/100 | Loss: 0.0266
Epoch 18/100 | Loss: 0.0253
Epoch 19/100 | Loss: 0.0257
Epoch 20/100 | Loss: 0.0249
Epoch 21/100 | Loss: 0.0247
Epoch 22/100 | Loss: 0.0238
Epoch 23/100 | Loss: 0.0244
Epoch 24/100 | Loss: 0.0236
Epoch 25/100 | Loss: 0.0233
Epoch 26/100 | Loss: 0.0241
Epoch 27/100 | Loss: 0.0236
Epoch 28/100 | Loss: 0.0222


KeyboardInterrupt: 